# Classifications of workers
## Goal:
    Create model for predict probobilty of leaving workers
## Tasks:
    1. Preprocesing data
    2. Analis data in power bi
    3. Create model

In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('aug_train.csv')

In [3]:
data.head()

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,8949,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevent experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevent experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0
3,33241,city_115,0.789,NaN,No relevent experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
4,666,city_162,0.767,Male,Has relevent experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0


Now we decode city to number values

In [4]:
data.city = data.city.map(lambda x: int(x.split('_')[1]))

In [5]:
data.loc[data.gender.isnull(),'gender'] = data.gender.mode()

In [6]:
data.dtypes

enrollee_id                 int64
city                        int64
city_development_index    float64
gender                     object
relevent_experience        object
enrolled_university        object
education_level            object
major_discipline           object
experience                 object
company_size               object
company_type               object
last_new_job               object
training_hours              int64
target                    float64
dtype: object

In [7]:
data.relevent_experience.value_counts()

Has relevent experience    13792
No relevent experience      5366
Name: relevent_experience, dtype: int64

In [8]:
enc_dict = {val:num for num, val in enumerate(data.relevent_experience.unique()[::-1])}
data.relevent_experience = data.relevent_experience.map(enc_dict)

In [9]:
data.relevent_experience.value_counts()

1    13792
0     5366
Name: relevent_experience, dtype: int64

In [10]:
data.loc[data.enrolled_university.isnull(),'enrolled_university'] = data.enrolled_university.mode()

In [11]:
data['enrolled_university'].value_counts()

no_enrollment       13817
Full time course     3757
Part time course     1198
Name: enrolled_university, dtype: int64

In [12]:
from sklearn.preprocessing import OrdinalEncoder

In [13]:
ord_enc = OrdinalEncoder()
data['enrolled_university'] = ord_enc.fit_transform(data[['relevent_experience']])

# Now i want use onehotencoding, but i not shure in this, if model will have a bad metrcis, i'm combed to this step 

In [14]:
data =  pd.concat([data, pd.get_dummies(data['education_level'])], axis=1)

In [15]:
data.drop('education_level', axis=1, inplace=True)

In [16]:
data.head()

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target,Graduate,High School,Masters,Phd,Primary School
0,8949,103,0.920,Male,1,1.0,STEM,>20,NaN,NaN,1,36,1.0,1,0,0,0,0
1,29725,40,0.776,Male,0,0.0,STEM,15,50-99,Pvt Ltd,>4,47,0.0,1,0,0,0,0
2,11561,21,0.624,NaN,0,0.0,STEM,5,NaN,NaN,never,83,0.0,1,0,0,0,0
3,33241,115,0.789,NaN,0,0.0,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0,1,0,0,0,0
4,666,162,0.767,Male,1,1.0,STEM,>20,50-99,Funded Startup,4,8,0.0,0,0,1,0,0


In [17]:
null_index = data.loc[data.major_discipline.isnull(), :].index

In [18]:
data.loc[null_index, 'major_discipline'] = 'STEM'

In [19]:
data= pd.concat([data, pd.get_dummies(data.major_discipline)], axis=1)
data.drop('major_discipline', axis=1, inplace=True)

In [20]:
pd.options.display.max_columns = 1000

In [21]:
data.head()

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,experience,company_size,company_type,last_new_job,training_hours,target,Graduate,High School,Masters,Phd,Primary School,Arts,Business Degree,Humanities,No Major,Other,STEM
0,8949,103,0.920,Male,1,1.0,>20,NaN,NaN,1,36,1.0,1,0,0,0,0,0,0,0,0,0,1
1,29725,40,0.776,Male,0,0.0,15,50-99,Pvt Ltd,>4,47,0.0,1,0,0,0,0,0,0,0,0,0,1
2,11561,21,0.624,NaN,0,0.0,5,NaN,NaN,never,83,0.0,1,0,0,0,0,0,0,0,0,0,1
3,33241,115,0.789,NaN,0,0.0,<1,NaN,Pvt Ltd,never,52,1.0,1,0,0,0,0,0,1,0,0,0,0
4,666,162,0.767,Male,1,1.0,>20,50-99,Funded Startup,4,8,0.0,0,0,1,0,0,0,0,0,0,0,1


In [22]:
data.experience.value_counts()

>20    3286
5      1430
4      1403
3      1354
6      1216
2      1127
7      1028
10      985
9       980
8       802
15      686
11      664
14      586
1       549
<1      522
16      508
12      494
13      399
17      342
19      304
18      280
20      148
Name: experience, dtype: int64

In [23]:
data.loc[data.experience=='<1', 'experience']=0
data.loc[data.experience=='>20', 'experience']=21

In [24]:
data.loc[data.experience.isnull(), 'experience'] = 9

In [25]:
data.experience = data.experience.astype(int)

In [26]:
data.company_size.value_counts()

50-99        3083
100-500      2571
10000+       2019
10/49        1471
1000-4999    1328
<10          1308
500-999       877
5000-9999     563
Name: company_size, dtype: int64

In [27]:
data.company_size.mode()

0    50-99
dtype: object

In [28]:
data.company_size.isnull().sum()

5938

In [29]:
data.loc[data.company_size.isnull(), 'company_size'] = '50-99' 

In [30]:
enc_dict = {val:num for num, val in enumerate(data.company_size.unique())}
data.company_size = data.company_size.map(enc_dict)

In [31]:
data.loc[data.company_type.isnull(), 'company_type'] = 'Pvt Ltd'
enc_dict = {val:num for num, val in enumerate(data.company_type.unique())}
data.company_type = data.company_type.map(enc_dict)

In [32]:
data.head()

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,experience,company_size,company_type,last_new_job,training_hours,target,Graduate,High School,Masters,Phd,Primary School,Arts,Business Degree,Humanities,No Major,Other,STEM
0,8949,103,0.920,Male,1,1.0,21,0,0,1,36,1.0,1,0,0,0,0,0,0,0,0,0,1
1,29725,40,0.776,Male,0,0.0,15,0,0,>4,47,0.0,1,0,0,0,0,0,0,0,0,0,1
2,11561,21,0.624,NaN,0,0.0,5,0,0,never,83,0.0,1,0,0,0,0,0,0,0,0,0,1
3,33241,115,0.789,NaN,0,0.0,0,0,0,never,52,1.0,1,0,0,0,0,0,1,0,0,0,0
4,666,162,0.767,Male,1,1.0,21,0,1,4,8,0.0,0,0,1,0,0,0,0,0,0,0,1


In [33]:
data.last_new_job.value_counts()
data.loc[data.last_new_job =='>4', 'last_new_job'] = 5
data.loc[data.last_new_job =='never', 'last_new_job'] =0

In [34]:
data.loc[data.last_new_job.isnull(), 'last_new_job'] = data.last_new_job.median()

In [35]:
data.loc[~data.gender.isnull(), 'gender'].unique()

array(['Male', 'Female', 'Other'], dtype=object)

In [36]:
enc_dict = {val:num for num, val in enumerate(data.loc[~data.gender.isnull(), 'gender'].unique())}
data.gender= data.gender.map(enc_dict)

In [37]:
train = data.loc[~data.gender.isnull(), :]
pred = data.loc[data.gender.isnull(), :]

In [38]:
pred.shape

(4508, 23)

In [39]:
y_train = train.gender
X_train = train.drop('gender', axis=1)
X_fpred = pred.drop('gender', axis=1)

In [40]:
X_fpred.shape

(4508, 22)

In [41]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

In [42]:
X_fpred.shape

(4508, 22)

In [43]:
std = StandardScaler()
X_train = std.fit_transform(X_train)
X_fpred = std.transform(X_fpred)

In [44]:
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)

LogisticRegression()

In [45]:
gender_pred = log_reg.predict(X_fpred)

In [46]:
gender_pred

array([0., 0., 0., ..., 0., 0., 0.])

In [47]:
data.loc[data.gender.isnull(), 'gender'] =gender_pred

In [48]:
data.last_new_job=data.last_new_job.astype(int)

In [49]:
data.isnull().sum()

enrollee_id               0
city                      0
city_development_index    0
gender                    0
relevent_experience       0
enrolled_university       0
experience                0
company_size              0
company_type              0
last_new_job              0
training_hours            0
target                    0
Graduate                  0
High School               0
Masters                   0
Phd                       0
Primary School            0
Arts                      0
Business Degree           0
Humanities                0
No Major                  0
Other                     0
STEM                      0
dtype: int64

## Okay. I finished prepare data, now i will try create a predicted model
Now i will crete RF for test 

In [50]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [51]:
def split_data(data):
    X = data.drop('target', axis=1)
    y = data.target
    X_train, X_test, y_train, y_test = train_test_split(X,y, stratify=y, test_size = 0.2)
    return {
        'X_train': X_train,
        'X_test': X_test,
        'y_train': y_train,
        'y_test': y_test
    }

In [52]:
D_M = split_data(data)

In [53]:
rand_forest = RandomForestClassifier()

In [54]:
rand_forest.fit(D_M['X_train'],D_M[ 'y_train'])

RandomForestClassifier()

I will check quality of model using presizyon and recall metrics 

In [88]:
from sklearn.metrics import precision_score, recall_score, accuracy_score

In [89]:
metrics = [precision_score, recall_score,accuracy_score]

In [57]:
print(precision_score.__name__)

precision_score


In [58]:
pred = rand_forest.predict(D_M['X_test'])
for metr in metrics:
    print(f"{metr.__name__} {metr(D_M['y_test'], pred)}")

precision_score 0.556998556998557
recall_score 0.4041884816753927


Result of metris so low, come back to step of prepare data

I think, in data a lot of parametrs have a zero value, now i check this

In [59]:
data["Graduate"].value_counts()[1]

11598

In [60]:
data.shape

(19158, 23)

In [61]:
for col in data.loc[:,'Graduate':].columns:
    one = data[col].value_counts()[1]
    print(f"{col} {one/data.shape[0]}")

Graduate 0.6053867835891011
High School 0.10528238855830463
Masters 0.22763336465184258
Phd 0.021609771374882555
Primary School 0.01607683474266625
Arts 0.013205971395761561
Business Degree 0.017068587535233323
Humanities 0.034920137801440654
No Major 0.01164004593381355
Other 0.019887253366739743
STEM 0.9032780039670112


I drop features, where quantity of one less than 20%

In [62]:
for col in data.loc[:,'Graduate':].columns:
    one = data[col].value_counts()[1]
    if one/data.shape[0]<0.2:
        data.drop(col, axis=1, inplace=True)

In [63]:
data.head()

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,experience,company_size,company_type,last_new_job,training_hours,target,Graduate,Masters,STEM
0,8949,103,0.920,0.0,1,1.0,21,0,0,1,36,1.0,1,0,1
1,29725,40,0.776,0.0,0,0.0,15,0,0,5,47,0.0,1,0,1
2,11561,21,0.624,0.0,0,0.0,5,0,0,0,83,0.0,1,0,1
3,33241,115,0.789,0.0,0,0.0,0,0,0,0,52,1.0,1,0,0
4,666,162,0.767,0.0,1,1.0,21,0,1,4,8,0.0,0,1,1


In [64]:
# data.drop(columns=['enrollee_id','city'], inplace=True)

In [65]:
data.head()

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,experience,company_size,company_type,last_new_job,training_hours,target,Graduate,Masters,STEM
0,8949,103,0.920,0.0,1,1.0,21,0,0,1,36,1.0,1,0,1
1,29725,40,0.776,0.0,0,0.0,15,0,0,5,47,0.0,1,0,1
2,11561,21,0.624,0.0,0,0.0,5,0,0,0,83,0.0,1,0,1
3,33241,115,0.789,0.0,0,0.0,0,0,0,0,52,1.0,1,0,0
4,666,162,0.767,0.0,1,1.0,21,0,1,4,8,0.0,0,1,1


Now i will try again fit model

In [66]:
New_data = split_data(data)

In [67]:
New_forest = RandomForestClassifier(class_weight={1:0.75, 0:0.25})
New_forest.fit(New_data['X_train'], New_data['y_train'])

RandomForestClassifier(class_weight={0: 0.25, 1: 0.75})

In [68]:
data.target.value_counts()[0]/data.shape[0], data.target.value_counts()[1]/data.shape[0]

(0.7506524689424784, 0.24934753105752167)

In [69]:
New_forest.score(y=New_data['y_test'], X=New_data['X_test'])

0.769572025052192

In [70]:
from xgboost import XGBClassifier

In [71]:
log_reg = LogisticRegression()
log_reg.fit(New_data['X_train'], New_data['y_train'])

LogisticRegression()

In [86]:
xgb = XGBClassifier()
xgb.fit(New_data['X_train'], New_data['y_train']);

[19:37:00] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [90]:
new_pred = xgb.predict(New_data['X_test'])
for metr in metrics:
    print(f"{metr.__name__} {metr(New_data['y_test'], new_pred)}")

precision_score 0.5640287769784172
recall_score 0.41047120418848165
accuracy_score 0.774008350730689


Now i will lookin for hipper parametrs